In [ ]:
import re
import simdjson as sj
import gzip
from tqdm import tqdm
import pandas as pd
from tqdm.notebook import tqdm
import swifter
import seaborn as sns
import numpy as np	
from nltk import sent_tokenize, word_tokenize
import re
from sentence_transformers import SentenceTransformer
from wutils.general import save_pickle, load_pickle
import fasttext
import contractions

In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [4]:
# model.encode(['test'])

In [ ]:
covid_re = re.compile(r'(covid|(?:corona)? ?virus)', re.IGNORECASE)
def fix_newlines(arr):
    output = []was it the 
    if isinstance(arr, float):
        print('WTF:', arr)
        return []
    for line in arr:
        output.extend(line.split('\n'))
    return output

In [ ]:
# pool = model.start_multi_process_pool()

In [11]:
in_file = gzip.open('./data/stage3_sitemap_crawl_mt.txt.gz')
bad_df = pd.read_json(in_file, lines=True)

In [13]:
bad_df['covidFreq'] = bad_df['cleanText'].swifter.apply(lambda t: len(covid_re.findall(t)))

/home/will/.local/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [14]:
bad_df = bad_df[bad_df['covidFreq'] >= 3]
bad_df['containsReuters'] = bad_df['cleanText'].swifter.apply(lambda x: 'Reuters' in x)
bad_df = bad_df[~bad_df['containsReuters']]

In [15]:
bad_df['sentences'] = bad_df['cleanText'].swifter.apply(sent_tokenize)

In [21]:
bad_df['fixed_sent'] = bad_df['sentences'].swifter.apply(fix_newlines)

WTF: nan



In [24]:
bad_df.dropna(inplace=True)

In [25]:
len(bad_df)

21854

In [ ]:
bad_df.loc['sbert'] = bad_df['sentences'].swifter.apply(lambda x: model.encode(x, convert_to_tensor=True, show_progress_bar=False))

In [28]:
converted = []
for sent in tqdm(bad_df['fixed_sent']):
    out = model.encode(sent, convert_to_tensor=True, show_progress_bar=False)
    converted.append(out)

In [39]:
bad_df['sent_embeddings'] = [None] * len(bad_df)

In [40]:
bad_df['sent_embeddings'] = [x.cpu().numpy() for x in converted]

In [45]:
save_pickle(bad_df, './pickles/newest_bad_df.pkl')

In [7]:
good_file = gzip.open('./data/stage3_all_real_processed_clean.txt.gz')
good_df = pd.read_json(good_file, lines=True)

In [8]:
# good_df = bad_df

In [7]:
len(good_df)

1599171

In [9]:
good_df['covidFreq'] = good_df['cleanText'].swifter.apply(lambda t: len(covid_re.findall(t)))

/home/will/.local/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
good_df = good_df[good_df['covidFreq'] >= 3]

In [11]:
len(good_df)

582711

In [11]:
sample_df = good_df.sample(n=100000, random_state=4242)

In [12]:
sample_df['sentences'] = sample_df['cleanText'].swifter.apply(sent_tokenize)
sample_df['fixed_sent'] = sample_df['sentences'].swifter.apply(fix_newlines)

In [13]:
converted = []
for sent in tqdm(sample_df['fixed_sent']):
    out = model.encode(sent, convert_to_tensor=True, show_progress_bar=False)
    converted.append(out.cpu().numpy())
sample_df['sent_embeddings'] = converted
save_pickle(sample_df, './pickles/newest_good_sample_df.pkl')

In [4]:
cord_df = load_pickle('biobert_encoded_cleaned_title_abstract_metadata_dataframe_2020_04_10.pkl')

In [57]:
def get_sents(x):
#     print(x.loc[x])
    output = []
    output.extend(sent_tokenize(x['title']))
    output.extend(sent_tokenize(x['abstract']))
    return output

In [6]:
cord_df['sentences'] = cord_df.swifter.apply(get_sents, axis=1)

/home/will/.local/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [7]:
converted = []
for sent in tqdm(cord_df['sentences']):
    out = model.encode(sent, convert_to_tensor=True, show_progress_bar=False)
    converted.append(out.cpu().numpy())
cord_df['sent_embeddings'] = converted
save_pickle(cord_df, './pickles/newest_good_cord_df.pkl')

Load from pickles

In [6]:
bad_df = load_pickle('./pickles/newest_bad_df.pkl')
good_df = load_pickle('./pickles/newest_good_sample_df.pkl')
cord_df = load_pickle('./pickles/newest_good_cord_df.pkl')

In [29]:
ft = fasttext.load_model('./data/cc.en.300.bin')

In [16]:
punct = set('')

In [82]:
bad_df['sent_embeddings'][15].shape

(22, 768)

In [25]:
def ft_embed_sent(sent):
    try:
        tokens = word_tokenize(contractions.fix(sent))
        keep = (x for x in tokens if any(y.isalnum() for y in x))
        return np.array([ft[x] for x in keep]).mean(axis=0)
    except Exception as e:
        print('error: ', e)
        return None

In [26]:
def embed_sents(sents):
    out = (ft_embed_sent(z) for z in sents)
    return [z for z in out if z is not None]

In [86]:
bad_df['ft_embeddings'] = bad_df['fixed_sent'].swifter.apply(lambda x: np.array([ft_embed_sent(y) for y in x]))

/home/will/.local/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [87]:
save_pickle(bad_df, './pickles/newest_bad_df_ft.pkl')

In [10]:
good_df['ft_embeddings'] = good_df['fixed_sent'].swifter.apply(embed_sents)

/home/will/.local/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


error:  string index out of range
error:  string index out of range
error:  string index out of range



In [11]:
save_pickle(good_df, './pickles/newest_good_sample_df_ft.pkl')

In [14]:
cord_df['ft_embeddings'] = cord_df['sentences'].swifter.apply(embed_sents)

/home/will/.local/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [15]:
save_pickle(cord_df, './pickles/newest_cord_df_ft.pkl')

In [55]:
not any(x.isalnum() for x in '.-')

True

In [71]:
pieces = word_tokenize(contractions.fix('Vote like you don\'t matter. He is an alchohol--happy person.'))
keep = [x for x in pieces if (any(y.isalnum() for y in x))]

In [72]:
keep

['Vote',
 'like',
 'you',
 'do',
 'not',
 'matter',
 'He',
 'is',
 'an',
 'alchohol',
 'happy',
 'person']

In [5]:
fact_df = load_pickle('./pickles/newest_claims_df_labelled.pkl')

In [11]:
fact_df['ft_embedding'] = fact_df['text'].swifter.apply(ft_embed_sent)

In [21]:
save_pickle(fact_df, './pickles/newest_claims_df_labelled.pkl')

In [16]:
fact_df['bert_embedding'] = fact_df['text'].swifter.apply(lambda x: model.encode(x, convert_to_tensor=True, show_progress_bar=False).cpu().numpy())

/home/will/.local/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## MM-COVID Dataset

In [6]:
import ujson

In [7]:
mm_data = []

with open('data/news_collection.json', 'r') as f:
    for line in f:
#         print(line)
        parsed = ujson.loads(line)
        if 'lang' in parsed and parsed['lang'] != 'en':
            continue
        if 'ref_source' not in parsed:
            continue
#         print(parsed.keys())
        mm_data.append({
            'label': parsed['label'],
            'news_id': parsed['news_id'],
            'url': parsed['ref_source_url'] if 'ref_source_url' in parsed else None,
            'statement': parsed['statement'] if 'statement' in parsed else None,
            'text': parsed['ref_source']['text']
        })
#         break
#         print(parsed['lang'])
#         break

In [8]:
mm_df = pd.DataFrame(mm_data)

In [9]:
from collections import Counter

In [10]:
mm_df['sent'] = mm_df['text'].swifter.apply(lambda x: [y for y in fix_newlines(sent_tokenize(x)) if y])

/home/will/.local/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
mm_df['bert'] = mm_df['sent'].swifter.apply(lambda x: model.encode(x, convert_to_tensor=True, show_progress_bar=False))

In [12]:
converted = []
for sent in tqdm(mm_df['sent']):
    if len(sent) == 0:
        converted.append(None)
        continue
    out = model.encode(sent, convert_to_tensor=True, show_progress_bar=False)
    converted.append(out)

In [19]:
mm_df['bert'] = [x.numpy() if x is not None else None for x in converted]

In [20]:
mm_df['mean_embed'] = mm_df['bert'].swifter.apply(lambda x: None if x is None else x.mean(axis=0))

In [30]:
mm_df['ft_embeddings'] = mm_df['sent'].swifter.apply(embed_sents)

In [32]:
mm_df['mean_ft'] = mm_df['ft_embeddings'].swifter.apply(lambda x: None if x is None else np.array(x).mean(axis=0))

/home/will/.local/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [40]:
Counter(mm_df['label'])

Counter({'real': 5325, 'fake': 2153})

In [36]:
save_pickle(mm_df, './pickles/mm_df.pkl')

In [53]:
who_cord_df = pd.read_csv('./data/metadata.csv')

/home/will/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
who_cord_df['journal'].value_counts()

bioRxiv                                            3498
BMJ                                                3340
PLoS One                                           3017
Journal of virology                                1899
Nature                                             1640
                                                   ... 
Progress in Industrial Mathematics at ECMI 2016       1
Information Processing & Management                   1
Behavioral neuroscience                               1
Adv. Integr. Med.                                     1
Diagnostic and Interventional Radiology               1
Name: journal, Length: 29620, dtype: int64

In [54]:
who_cord_df = who_cord_df[who_cord_df['source_x'] == 'WHO']

In [55]:
who_cord_df = who_cord_df.drop(columns=['sha', 'doi', 'pmcid', 'pubmed_id', 'license', 'publish_time', 'journal', 'mag_id', 'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files', 'url', 's2_id'])

In [75]:
who_cord_df = who_cord_df.dropna()

In [76]:
def get_sents(x):
#     print(x.loc[x])
    output = []
#     print(x['abstract'])
#     print(sent_tokenize(x['abstract']))
    output.extend(sent_tokenize(x['title']))
    output.extend(sent_tokenize(x['abstract']))
    return output

In [77]:
who_cord_df['sentences'] = who_cord_df.swifter.apply(get_sents, axis=1)

/home/will/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
converted = []
for sent in tqdm(who_cord_df['sentences']):
    out = model.encode(sent, convert_to_tensor=True, show_progress_bar=False)
    converted.append(out.cpu().numpy())
who_cord_df['sent_embeddings'] = converted

/home/will/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [80]:
who_cord_df['ft_embeddings'] = who_cord_df['sentences'].swifter.apply(embed_sents)

/home/will/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
save_pickle(who_cord_df, './pickles/who_cord_df.pkl')